# Practical Tuesday Afternoon (Part 1): Finetuning LLMs

This notebook aims to give a quick overview of modern finetuning techniques on lower end GPUs. The goal is NOT to reinvent the wheel, but rather give an overview of several current libs and good practices. Lots of concepts are in there - this notebook does not aim to explain them. This is what the TAs are for, and Google can help too !



It touches up on several key concepts:

1.  Instruction Fine-tuning a SLM (Part 1)
  *  Base vs Instruct/Chat models
  *  LoRA Adapters
  *  Unsloth
  *  Quantization
  *  Chat templating and data preparation
  *  SFTTrainers
  *  Going further: Preparing your own finetuning dataset through synthetic data generation.

2. Retrieval Augmented Generation with Vision Language Models (Part 2)
  * Check out `Practical Tuesday Afternoon (Part 2)`


<a target="_blank" href="https://colab.research.google.com/github/https://colab.research.google.com/drive/1JOhf2QCTeV7-3JHsGTOCxyn4mIrVCsKI?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook is inspired by ressources from Unsloth AI, Manuel Faysse.

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Try out the base model

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)


messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The first two numbers are 1 and 1, and the next number is the sum of the two preceding numbers. The sequence is named after Leonardo of Pisa, also known as Fibonacci, who popularized the sequence by studying the growth of a certain type of snail. The sequence is also known as the Lucas sequence when the first two terms are 2 and 1, and the next number is the sum of the two preceding numbers.

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The first two numbers are 1 and 1, and the next number is the sum of the two preceding numbers. The sequence is named after Leonardo of Pisa, also known as Fibonacci, who popularized the sequence by studying the growth of a certain type of snail. The sequence is also known as the Lucas sequence when the first two terms are 2 and 1, and the next number is the sum of the two preceding numbers.

The base model has internal knowledge. However, it is not trained to respond to instructions and has tendencies to loop around...

**To fix this, we can "Instruction Finetune" the model !**

We will train the model using Low-rank adapters to reduce memory requirements !

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
# subsample dataset for speed
dataset = dataset.shuffle(seed = 3407).select(range(5000))

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

[{'content': 'Explain how the principle of non-contradiction can be applied to prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, and use this principle to demonstrate the truth or falsity of the given statement.',
  'role': 'user'},
 {'content': "The principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller than p, they cannot be equal to p. Therefore, they must be less than p. \nBut since p is a prime number, it can only be divided by 1 and itself. Therefore, neither q nor r can divide p. \nNow, l

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain how the principle of non-contradiction can be applied to prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, and use this principle to demonstrate the truth or falsity of the given statement.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

max_steps is given, it will override any value given in num_train_epochs


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain how the principle of non-contradiction can be applied to prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, and use this principle to demonstrate the truth or falsity of the given statement.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller 

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                      \n\nThe principle of non-contradiction states that a statement and its negation cannot both be true at the same time. \nTo prove that any prime number greater than 2 cannot be expressed as the sum of two smaller prime numbers, we can assume the opposite and show that it leads to a contradiction. \nLet's assume that there exists a prime number p greater than 2 that can be expressed as the sum of two smaller prime numbers, say p = q + r, where q and r are smaller prime numbers. \nSince q and r are smaller than p, they cannot be equal to p. Therefore, they must be less than p. \nBut since p is a prime number, it can only be divided by 1 and itself. Therefore, neither q nor r can divide p. \nNow, let's consider the number (p-2). Since p is greater than 2, we know that (p-2) is a positive integer. \nWe can express (p-2) as the sum of q and r: (p-2) = q + r. \nAdding 2 to both sides, we get: p = (q+2) + (r

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
10.578 GB of memory reserved.


In [ ]:
# This is a very reduced training time for the practical - don't expect optimal results !
# What constitutes "Optimal" training is not evident, but in general we want at least a few thousand high-quality and diverse samples !
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


Step,Training Loss
1,0.949700
2,1.023000
3,0.890000
4,0.952700
5,0.858200
6,1.220800
7,0.862500
8,1.010600
9,0.888500
10,0.941200


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

497.8748 seconds used for training.
8.3 minutes used for training.
Peak reserved memory = 10.578 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 71.725 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 200, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The sequence starts with 1 and 1, and each subsequent number is the sum of the two preceding numbers.\n\nTo continue the Fibonacci sequence, you can use the following formula:\n\nF(n) = F(n-1) + F(n-2)\n\nwhere F(n) is the n-th number in the sequence.\n\nFor example, to get the next number in the sequence after 8, you can use the formula:\n\nF(9) = F(8) + F(7)\n\nF(9) = 8 + 5\n\nF(9) = 13\n\nSo, the next number in the sequence after 8 is 13.\r\r\n\r\r\n—\n\nThe Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The sequence starts with 1 and 1,

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 200,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The sequence starts with 1 and 1, and each subsequent number is the sum of the two preceding numbers.

To continue the Fibonacci sequence, you can use the following formula:

F(n) = F(n-1) + F(n-2)

where F(n) is the n-th number in the sequence.

For example, to get the next number in the sequence after 8, you can use the formula:

F(9) = F(8) + F(7)

F(9) = 8 + 5

F(9) = 13

So, the next number in the sequence after 8 is 13.

—

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The sequence starts with 1 and 1, and each subsequent number is the sum of the two preceding numbers.

To continue


The model now seems to answer the question !
It does not seem to stop at the end... This implies it does not generate its `<|eot_id|>`... This may be due to insufficient training, or lack of short answers in the training dataset !

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the capital city of France, Paris, is a popular tourist attraction and one of the most recognizable landmarks in the world. Standing at a height of 324 meters (1,063 feet), it is the tallest structure in France and the second tallest structure in the world. The tower was built in 1889 as a part of the World's Fair, and it was originally intended to be a temporary structure. However, it has become a permanent feature of the Paris skyline and a symbol of the city's identity.

The Eiffel Tower is made up of three levels, with the first level being


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

Credits to Unsloth AI for the amazing work facilitating training such models.

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
</div>

## (Optional) Your turn to play !

For the moment, we ran a very standard instruction tuning procedure to make a generalist chat model... But possibilities are endless... You could specialize the model on whatever you want with a good dataset - writing haikus, responding in French, even being better at coding !

How do we create this datasets ? Costly human annotation, or nowadays, using other bigger LLMs (weak distillation).

If you want, go ahead and use an API (Groq, OpenAI, Claude) or load a bigger model and generate instructions with it to create an instruction dataset for the task YOU want to optimize in the model !

*Note: The better a model is, the less it needs to be finetuned and the better it will be able to follow prompts you give it*


**Getting started**

For the moment, Groq provides free API access to LLama3 70B:

https://console.groq.com/


Optionnally, feel free to create an account and test this out.

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Once you have your API access set up:


*   Work on your prompting strategy to get high quality data with good diversity
    * Hint: You can use an LLM to judge on whether if the generated sample is good !
    * Try to get at least ~500 samples
* Format your dataset in the correct chat format (see Data Prep part)
    * Hint: If your dataset is a bit too small, you can start training from an instruct model instead of a base model and prompt it so that it kind of "understands" what you are aiming to do with less training data
* Train it using techniques learnt in this tutorial
* Evaluate your trained model
   * Play with it
   * You can use LLM-as-a-judge to grade the quality of your model's outputs !



## Conclusion

This is the end of Part 1 of this practical on finetuning LLMs.
In part 2, we will use small instruction models for Vision RAG !